In [5]:
import torch
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [23]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [17]:
version_9=pd.read_csv(r'C:\Users\sim\OneDrive\바탕 화면\성균관대 인공지능 프로젝\version09\version09.csv')

In [18]:
version_9.head()

Unnamed: 0          방송일시  노출(분)    마더코드    상품코드   상품명  상품군  브랜드   판매단가  \
0           0  1.546320e+18     20  100346  201072  1592    7  373  39900   
1           1  1.546320e+18     20  100346  201079  1598    7  373  39900   
2           2  1.546320e+18     20  100346  201072  1592    7  373  39900   
3           3  1.546320e+18     20  100346  201079  1598    7  373  39900   
4           4  1.546320e+18     20  100346  201072  1592    7  373  39900   

      유동인구      취급액  
0  8123838  2099000  
1  8123838  4371000  
2  8123838  3262000  
3  8123838  6955000  
4  8123838  6672000

In [19]:
version_9.describe()

Unnamed: 0          방송일시         노출(분)           마더코드           상품코드  \
count  38309.000000  3.830900e+04  38309.000000   38309.000000   38309.000000   
mean   19154.000000  1.562024e+18     20.438774  100375.449268  201215.751651   
std    11059.000068  8.918563e+15      3.416357     245.281352     736.923249   
min        0.000000  1.546320e+18      2.000000  100000.000000  200000.000000   
25%     9577.000000  1.554500e+18     20.000000  100148.000000  200541.000000   
50%    19154.000000  1.561920e+18     20.000000  100323.000000  201164.000000   
75%    28731.000000  1.569670e+18     20.000000  100572.000000  201861.000000   
max    38308.000000  1.577840e+18     60.000000  100848.000000  202512.000000   

                상품명           상품군           브랜드          판매단가          유동인구  \
count  38309.000000  38309.000000  38309.000000  3.830900e+04  3.830900e+04   
mean     910.343888      5.497298    208.485134  4.565537e+05  8.137375e+06   
std      502.750975      3.493707    133.082950  7.260555e+05  4.258952e+05   
min        0.000000      0.000000      0.000000  0.000000e+00  1.369260e+06   
25%      520.000000      2.000000     92.000000  5.900000e+04  8.000900e+06   
50%      883.000000      6.000000    192.000000  1.090000e+05  8.123838e+06   
75%     1328.000000      9.000000    346.000000  4.990000e+05  8.360840e+06   
max     1769.000000     11.000000    420.000000  7.930000e+06  8.733140e+06   

                취급액  
count  3.830900e+04  
mean   2.133546e+07  
std    2.022987e+07  
min    0.000000e+00  
25%    6.383000e+06  
50%    1.552800e+07  
75%    3.106700e+07  
max    3.220090e+08

In [68]:
seq_length=len(version_9)
data_dim = 11
hidden_dim = 14 #상시 변동 가능
output_dim =1
learning_rate =0.01
iterations = 500
#hidden dimension 어떻게 설정해야 할 지 모르겠다

In [69]:
def minmax_scaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / (denominator + 1e-7)

#수정예정

In [70]:
def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(time_series) - seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i + seq_length, [-1]]
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

In [79]:
# load data
xy = pd.read_csv(r'C:\Users\sim\OneDrive\바탕 화면\성균관대 인공지능 프로젝\version09\version09.csv' ,encoding='UTF-8')
xy = xy[::-1] 

train_size = int(len(xy) * 0.7)
train_set = xy[0:train_size]
test_set = xy[train_size - seq_length:]

train_set = minmax_scaler(train_set)
test_set = minmax_scaler(test_set)

trainX, trainY = build_dataset(train_set, seq_length)
testX, testY = build_dataset(test_set, seq_length)

trainX_tensor = torch.FloatTensor(trainX)
trainY_tensor = torch.FloatTensor(trainY)

testX_tensor = torch.FloatTensor(testX)
testY_tensor = torch.FloatTensor(testY)


In [72]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.LSTM(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x[:, -1])
        return x


net = Net(data_dim, hidden_dim, output_dim, 1)

In [73]:
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [67]:
for i in range(iterations):

    optimizer.zero_grad()
    outputs = net(trainX_tensor)
    loss = criterion(outputs, trainY_tensor)
    loss.backward()
    optimizer.step()
    print(i, loss.item())
    

    
#runtime error input must have 3dimensions, but 1

RuntimeError: input must have 3 dimensions, got 1